In [28]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor, VecFrameStack, DummyVecEnv

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback

def make_procgen_env(env_name, num_envs=1, start_level=0, num_levels=0):
    def _init():
        env = gym.make(env_name, start_level=start_level, num_levels=num_levels, render_mode="rgb_array")
        return env
    return DummyVecEnv([_init] * num_envs)

env_name = "procgen:procgen-heist-v0"  
num_envs = 4 
learning_rate = 3e-4
n_steps = 2048
batch_size = 64
n_epochs = 10
gamma = 0.99

env = make_procgen_env(env_name, num_envs)
env = VecMonitor(env)
env = VecFrameStack(env, n_stack=4)





/Users/bensturgeon/Library/Caches/pypoetry/virtualenvs/ai-safety-camp-Y8XZewIj-py3.10/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [9]:
import torch

# Load the model
model = torch.load('../heist_model_final_easy.pt', map_location=torch.device('cpu'))

print("State_dict:")
for param_tensor in model:
    print(param_tensor, "\t", model[param_tensor].size())

State_dict:
conv_seqs.0.conv.weight 	 torch.Size([16, 3, 3, 3])
conv_seqs.0.conv.bias 	 torch.Size([16])
conv_seqs.0.res_block0.conv0.weight 	 torch.Size([16, 16, 3, 3])
conv_seqs.0.res_block0.conv0.bias 	 torch.Size([16])
conv_seqs.0.res_block0.conv1.weight 	 torch.Size([16, 16, 3, 3])
conv_seqs.0.res_block0.conv1.bias 	 torch.Size([16])
conv_seqs.0.res_block1.conv0.weight 	 torch.Size([16, 16, 3, 3])
conv_seqs.0.res_block1.conv0.bias 	 torch.Size([16])
conv_seqs.0.res_block1.conv1.weight 	 torch.Size([16, 16, 3, 3])
conv_seqs.0.res_block1.conv1.bias 	 torch.Size([16])
conv_seqs.1.conv.weight 	 torch.Size([32, 16, 3, 3])
conv_seqs.1.conv.bias 	 torch.Size([32])
conv_seqs.1.res_block0.conv0.weight 	 torch.Size([32, 32, 3, 3])
conv_seqs.1.res_block0.conv0.bias 	 torch.Size([32])
conv_seqs.1.res_block0.conv1.weight 	 torch.Size([32, 32, 3, 3])
conv_seqs.1.res_block0.conv1.bias 	 torch.Size([32])
conv_seqs.1.res_block1.conv0.weight 	 torch.Size([32, 32, 3, 3])
conv_seqs.1.res_block1.conv0

In [29]:
import sys
sys.path.append('../') 

from src.policies import ImpalaCNN

observation_space = env.observation_space

action_space = env.action_space.n

In [30]:


model_struct = ImpalaCNN(observation_space, action_space)



In [31]:
model_struct.state_dict = model


In [44]:
state = env.reset()
obs, reward, done, information = env.step((1,))



IndexError: tuple index out of range

In [34]:
env.action_space.sample()

9